## 1. Installing GEE

This is working with conda.\
For installing conda: https://developers.google.com/earth-engine/python_install-conda

#### Create a new environment (recommended):

In [ ]:
#conda create --name ee
#conda activate ee

#### Install the earthengine api

In [ ]:
#conda install -c conda-forge earthengine-api

#### Authenticate account (need it for the first time of use)

In [ ]:
#earthengine authenticate

In [29]:
import sys
print(sys.path)

['/home/jovyan/project_notebooks', '/srv/conda/envs/notebook/lib/python37.zip', '/srv/conda/envs/notebook/lib/python3.7', '/srv/conda/envs/notebook/lib/python3.7/lib-dynload', '', '/srv/conda/envs/notebook/lib/python3.7/site-packages', '/srv/conda/envs/notebook/lib/python3.7/site-packages/IPython/extensions', '/home/jovyan/.ipython']


In [42]:
cd /home/jovyan

/home/jovyan


In [45]:
find / -type d -name 'earthengine'

SyntaxError: invalid syntax (3411310842.py, line 1)

How to authenticate this server for accessing Earth Engine\
Step 1 - Click on the following link\
[Link to generate an authentication code for accessing Earth Engine](https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine.readonly&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com) \

Step 2 - Click on Accept\
Step 3 - Copy the authentication code that is returned\
Step 4 - Paste the authentication code below, then run the code by pressing the play button\

In [36]:
authentication_code ='PASTE_CODE_HERE'

import ee
import errno
import json
import os
import urllib

from ee import oauth as OAuthInfo

# Try to initialize Earth Engine, and if unsuccessful try to get a credentials file
# using the authentication code provided above.
try:
    ee.Initialize()
except:
    token_request_params = {
      'code': authentication_code,
      'client_id': OAuthInfo.CLIENT_ID,
      'client_secret': OAuthInfo.CLIENT_SECRET,
      'redirect_uri': 'urn:ietf:wg:oauth:2.0:oob',
      'grant_type': 'authorization_code'
    }
    refresh_token = None
    try:
        response = urllib.request.urlopen('https://accounts.google.com/o/oauth2/token',
                               urllib.parse.urlencode(token_request_params)).read()
        tokens = json.loads(response)
        refresh_token = tokens['refresh_token']
    except urllib.error.HTTPError as e:
        raise Exception('Problem requesting tokens.  Please try again.  %s %s' %
                    (e, e.read()))

    ### Write refresh token to filesystem for later use
    credentials_path = OAuthInfo.credentials_path()
    dirname = os.path.dirname(credentials_path)
    try:
        os.makedirs(dirname)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise Exception('Error creating %s: %s' % (dirname, e))

    json.dump({'refresh_token': refresh_token}, open(credentials_path, 'w'))

    print('\nSuccessfully saved authorization to %s' % credentials_path)
    
# Try to authenticate to Earth Engine.
try:
  ee.Initialize()
  print('\nSuccessfully authenticated to Earth Engine!')
except:
  print('\nOops. Something went wrong!')

TypeError: POST data should be bytes, an iterable of bytes, or a file object. It cannot be of type str.

## 2. Working with GEE

#### Import EE

In [ ]:
import ee
ee.Initialize()

In [3]:
## Verify the ee module is loaded.
print(ee.__version__)

0.1.283


#### Install other useful packages:

In [ ]:
pip install geemap ##Package for visualizing geospatial data.

#### Load satellite dataset: https://developers.google.com/earth-engine/datasets/

In [ ]:
collection = ee.ImageCollection("COPERNICUS/S2_SR") ##Sentinel-2 L2A

#### Filter collection by date, region of interest and cloud coverage:

In [ ]:
## Some settings:
start_date = '2020-01-01'
end_date = '2020-12-31'
cloud_cover = 30
states = ee.FeatureCollection("TIGER/2018/States") ## US administrative division
aoi = states.filterMetadata('NAME','equals','Florida') ## Select the state of Florida.

## Filter collection:
filtered = collection.filterDate(start_date,end_date) \
            .filterBounds(aoi) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover))

In [ ]:
## Check number of images loaded:
print(filtered.size().getInfo())

#### Clip collection and apply cloud mask:

In [ ]:
## Function to clip an image collection
def clipCollection(img):
    return img.clip(aoi)

## Function to mask clouds using the Sentinel-2 QA band.
def maskS2clouds(image):
    qa = image.select('QA60')

    ## Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;

    ## Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    ## Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000) \
    .select("B.*") \
    .copyProperties(image, ["system:time_start"])

In [ ]:
## Run functions and create mosaic:
mosaic = filtered.map(maskS2clouds).map(clipCollection).median()

#### Visualize map

In [ ]:
## Visualization parameters:
viz = {
    'min': 0,
    'max': 2.0,
    'bands': ['B4','B3','B2'],
    'gamma': 1.5}

# initialize our map
Map = geemap.Map()
Map.centerObject(aoi, 9)
Map.addLayer(mosaic, viz, "Florida")

Map.addLayerControl()
Map